In [1]:
# --- 1. PREPARACIÓN DEL ENTORNO ---
import os
import shutil
import hashlib
import re
import string
import unicodedata
import pandas as pd
import nltk
import fitz  # PyMuPDF
import joblib

print("Cargando recursos de lenguaje (NLTK)...")
# Descargar y definir las 'stopwords'
try:
    nltk.data.find('corpora/stopwords')
except nltk.downloader.DownloadError:
    nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('spanish'))


# --- 2. CONFIGURACIÓN DEL SCRIPT ---
# --- ¡IMPORTANTE! MODIFICA ESTAS VARIABLES SEGÚN TUS NECESIDADES ---

# Carpeta que contiene los nuevos informes en PDF que quieres clasificar
CARPETA_NUEVOS_PDFS = 'Informes_Para_Testear'

# Nombre del archivo de tu modelo entrenado (el mejor que tengas)
NOMBRE_MODELO_GUARDADO = 'pipeline_naive_bayes.pkl' 

# Nombre de la carpeta temporal que se creará para los archivos intermedios
CARPETA_TRABAJO_TEMPORAL = 'temp_processing'

# Nombre del archivo final donde se guardarán los resultados
ARCHIVO_RESULTADOS_CSV = 'resultados_finales.csv'


# --- 3. DEFINICIÓN DE TODAS LAS FUNCIONES NECESARIAS ---

def extraer_texto_de_pdf(ruta_pdf):
    try:
        doc = fitz.open(ruta_pdf)
        texto = "".join(page.get_text() for page in doc)
        doc.close()
        return texto
    except Exception:
        return ""

def calcular_hash(ruta_archivo):
    sha256_hash = hashlib.sha256()
    try:
        with open(ruta_archivo, "rb") as f:
            for byte_block in iter(lambda: f.read(4096), b""):
                sha256_hash.update(byte_block)
        return sha256_hash.hexdigest()
    except IOError:
        return None

def anonimizar_informe(texto_completo):
    marcador_inicio = r"Fecha Examen"
    marcador_fin = r"Dr(\(a\))?\.|\\bAtentamente\\b"
    match_inicio = re.search(marcador_inicio, texto_completo, re.IGNORECASE)
    match_fin = re.search(marcador_fin, texto_completo, re.IGNORECASE)
    if match_inicio and match_fin:
        cuerpo = texto_completo[match_inicio.start():match_fin.start()]
        encabezado_anon = "[BLOQUE PACIENTE ANONIMIZADO]\\n"
        pie_pagina_anon = "\\n[BLOQUE MÉDICO ANONIMIZADO]"
        cuerpo_anon = re.sub(r'\\b\\d{1,2}\\.?\\d{3}\\.?\\d{3}-[\\dkK]\\b', '[ID_ANONIMIZADO]', cuerpo)
        return encabezado_anon + cuerpo_anon + pie_pagina_anon
    return texto_completo

def remover_acentos(texto):
    forma_nfd = unicodedata.normalize('NFD', texto)
    return "".join([c for c in forma_nfd if not unicodedata.combining(c)])

def preprocess_text(text):
    text = text.lower()
    text = remover_acentos(text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return " ".join(tokens)


# --- 4. FUNCIÓN PRINCIPAL QUE ORQUESTA TODO EL PROCESO ---

def clasificar_lote_pdf(config):
    """
    Función principal que ejecuta todo el pipeline de clasificación.
    """
    # Crear carpetas temporales para organizar el trabajo
    path_texto_crudo = os.path.join(config['temp_folder'], '1_texto_crudo')
    path_unicos = os.path.join(config['temp_folder'], '2_unicos')
    path_anonimizados = os.path.join(config['temp_folder'], '3_anonimizados')
    os.makedirs(path_texto_crudo, exist_ok=True)
    os.makedirs(path_unicos, exist_ok=True)
    os.makedirs(path_anonimizados, exist_ok=True)

    # --- PASO 1: PDF a Texto ---
    print(f"\\n--- PASO 1: Convirtiendo PDFs de '{config['input_folder']}' a Texto ---")
    for fname in os.listdir(config['input_folder']):
        if fname.lower().endswith(".pdf"):
            ruta_pdf = os.path.join(config['input_folder'], fname)
            texto = extraer_texto_de_pdf(ruta_pdf)
            if texto:
                ruta_txt = os.path.join(path_texto_crudo, os.path.splitext(fname)[0] + '.txt')
                with open(ruta_txt, 'w', encoding='utf-8') as f:
                    f.write(texto)
    print("Conversión a texto completada.")

    # --- PASO 2: Eliminar Duplicados ---
    print(f"\\n--- PASO 2: Buscando y filtrando informes duplicados ---")
    hashes = {}
    for fname in os.listdir(path_texto_crudo):
        ruta_completa = os.path.join(path_texto_crudo, fname)
        file_hash = calcular_hash(ruta_completa)
        if file_hash not in hashes:
            hashes[file_hash] = ruta_completa
    
    for ruta_original in hashes.values():
        shutil.copy2(ruta_original, path_unicos)
    print(f"Se encontraron {len(hashes)} informes únicos para procesar.")

    # --- PASO 3: Anonimizar Textos Únicos ---
    print(f"\\n--- PASO 3: Anonimizando informes ---")
    for fname in os.listdir(path_unicos):
        ruta_txt_unico = os.path.join(path_unicos, fname)
        with open(ruta_txt_unico, 'r', encoding='utf-8') as f:
            texto_original = f.read()
        
        texto_anon = anonimizar_informe(texto_original)
        
        ruta_txt_anon = os.path.join(path_anonimizados, fname)
        with open(ruta_txt_anon, 'w', encoding='utf-8') as f:
            f.write(texto_anon)
    print("Anonimización completada.")

    # --- PASO 4: Cargar Modelo y Clasificar ---
    print(f"\\n--- PASO 4: Cargando modelo y clasificando ---")
    try:
        pipeline_modelo = joblib.load(config['model_path'])
    except FileNotFoundError:
        print(f"¡ERROR! No se encontró el archivo del modelo: {config['model_path']}")
        return

    resultados = []
    for fname in os.listdir(path_anonimizados):
        ruta_txt_final = os.path.join(path_anonimizados, fname)
        with open(ruta_txt_final, 'r', encoding='utf-8') as f:
            texto_a_clasificar = f.read()
            
        texto_limpio = preprocess_text(texto_a_clasificar)
        
        # Predicción
        prediccion = pipeline_modelo.predict([texto_limpio])[0]
        probabilidades = pipeline_modelo.predict_proba([texto_limpio])[0]
        
        etiqueta = "Crítico" if prediccion == 1 else "No Crítico"
        prob_critico = probabilidades[1]
        
        print(f"  - Informe: {fname} -> Resultado: {etiqueta} (Certeza: {prob_critico:.2%})")
        resultados.append({'Archivo': fname, 'Clasificacion': etiqueta, 'Probabilidad_Critico': prob_critico})

    # --- PASO 5: Guardar Resultados ---
    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_csv(config['output_csv'], index=False)
    print(f"\\n✅ ¡Proceso finalizado! Resultados guardados en '{config['output_csv']}'")

    # --- PASO 6: Limpieza ---
    shutil.rmtree(config['temp_folder'])
    print(f"Carpeta temporal '{config['temp_folder']}' eliminada.")


# --- 5. EJECUCIÓN DEL SCRIPT ---
if __name__ == "__main__":
    config = {
        'input_folder': CARPETA_NUEVOS_PDFS,
        'model_path': NOMBRE_MODELO_GUARDADO,
        'temp_folder': CARPETA_TRABAJO_TEMPORAL,
        'output_csv': ARCHIVO_RESULTADOS_CSV
    }
    clasificar_lote_pdf(config)

Cargando recursos de lenguaje (NLTK)...
\n--- PASO 1: Convirtiendo PDFs de 'Informes_Para_Testear' a Texto ---
Conversión a texto completada.
\n--- PASO 2: Buscando y filtrando informes duplicados ---
Se encontraron 2 informes únicos para procesar.
\n--- PASO 3: Anonimizando informes ---
Anonimización completada.
\n--- PASO 4: Cargando modelo y clasificando ---
  - Informe: resultadoexamen (1).txt -> Resultado: No Crítico (Certeza: 23.46%)
  - Informe: resultadoexamen.txt -> Resultado: Crítico (Certeza: 79.56%)
\n✅ ¡Proceso finalizado! Resultados guardados en 'resultados_finales.csv'
Carpeta temporal 'temp_processing' eliminada.
